# Assignment 5, Question 6: Data Transformation

**Points: 20**

Transform and engineer features from the clinical trial dataset.

## Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import utilities
from q3_data_utils import load_data, clean_data, transform_types, create_bins, fill_missing

df = load_data('output/q5_cleaned_data.csv')
print(f"Loaded {len(df)} patients")

# Prewritten visualization functions for transformation analysis
def plot_distribution(series, title, figsize=(10, 6)):
    """
    Create a histogram of a numeric series.
    
    Args:
        series: pandas Series with numeric data
        title: Chart title
        figsize: Figure size tuple
    """
    plt.figure(figsize=figsize)
    series.hist(bins=30)
    plt.title(title)
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.tight_layout()
    plt.show()

def plot_value_counts(series, title, figsize=(10, 6)):
    """
    Create a bar chart of value counts.
    
    Args:
        series: pandas Series with value counts
        title: Chart title
        figsize: Figure size tuple
    """
    plt.figure(figsize=figsize)
    series.plot(kind='bar')
    plt.title(title)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

Loaded 10000 patients


## Part 1: Type Conversions (5 points)

1. Convert 'enrollment_date' to datetime using the `transform_types()` utility
2. Convert categorical columns ('site', 'intervention_group', 'sex') to category dtype
3. Ensure all numeric columns are proper numeric types
4. Display the updated dtypes

In [2]:
# Type conversions
# 1. Use transform_types() to convert enrollment_date to datetime
# 2. Convert categorical columns ('site', 'intervention_group', 'sex') to category dtype
# 3. Ensure all numeric columns are proper numeric types
df = transform_types(df, {'enrollment_date': 'datetime', 'site': 'category', 'intervention_group': 'category', 'sex': 'category'})

# 4. Display the updated dtypes using df.dtypes
print(df.dtypes)


patient_id                    object
age                          float64
sex                         category
bmi                          float64
enrollment_date       datetime64[ns]
systolic_bp                  float64
diastolic_bp                 float64
cholesterol_total            float64
cholesterol_hdl              float64
cholesterol_ldl              float64
glucose_fasting              float64
site                        category
intervention_group          category
follow_up_months               int64
adverse_events                 int64
outcome_cvd                   object
adherence_pct                float64
dropout                       object
dtype: object


## Part 2: Feature Engineering (8 points)

Create these new calculated columns:

1. `cholesterol_ratio` = cholesterol_ldl / cholesterol_hdl
2. `bp_category` = categorize systolic BP:
   - 'Normal': < 120
   - 'Elevated': 120-129
   - 'High': >= 130
3. `age_group` using `create_bins()` utility:
   - Bins: [0, 40, 55, 70, 100]
   - Labels: ['<40', '40-54', '55-69', '70+']
4. `bmi_category` using standard BMI categories:
   - Underweight: <18.5
   - Normal: 18.5-24.9
   - Overweight: 25-29.9
   - Obese: >=30

In [3]:
# Create cholesterol ratio
df['cholesterol_ratio'] = df['cholesterol_ldl'] / df['cholesterol_hdl']
print(df[['cholesterol_ldl', 'cholesterol_hdl', 'cholesterol_ratio']].head())

   cholesterol_ldl  cholesterol_hdl  cholesterol_ratio
0             41.0             55.0           0.745455
1            107.0             58.0           1.844828
2             82.0             56.0           1.464286
3            104.0             56.0           1.857143
4             75.0             78.0           0.961538


In [4]:
# Categorize blood pressure
# Define bins and labels
bins = [0, 120, 130, np.inf]  # 0-119, 120-129, 130+
labels = ['Normal', 'Elevated', 'High']

df = create_bins(df, column='systolic_bp', bins=bins, labels=labels, new_column='bp_category')
print(df[['systolic_bp', 'bp_category']].head())

   systolic_bp bp_category
0        123.0    Elevated
1        139.0        High
2        123.0    Elevated
3        116.0      Normal
4         97.0      Normal


**Note:** The `create_bins()` function has an optional `new_column` parameter. If you don't specify it, the new column will be named `{original_column}_binned`. You can use `new_column='age_group'` to give it a custom name.


In [5]:
# Create age groups
# `age_group`
bins = [0, 40, 55, 70, np.inf]
labels = ['<40', '40-54', '55-69', '70+']

df = create_bins(df, column='age', bins=bins, labels=labels, new_column='age_group')
print(df[['age', 'age_group']].head())

    age age_group
0  80.0       70+
1  80.0       70+
2  82.0       70+
3  95.0       70+
4  95.0       70+


In [6]:
# Create BMI categories
# `bmi_category`
bins = [0, 18.5, 25, 30, np.inf]
labels = ['Underweight', 'Normal', 'Overweight', 'Obese']

df = create_bins(df, column='bmi', bins=bins, labels=labels, new_column='bmi_category')
print(df[['bmi', 'bmi_category']].head())

    bmi bmi_category
0  29.3   Overweight
1  26.2   Overweight
2  26.2   Overweight
3  25.4   Overweight
4  26.2   Overweight


## Part 3: String Cleaning (2 points)

If there are any string columns that need cleaning:
1. Convert to lowercase
2. Strip whitespace
3. Replace any placeholder values

In [7]:
# TODO: String cleaning

# Clean up "site" column
df['site'] = (
    df['site'].astype(str)
    .str.replace('_', ' ', regex=False)
    .str.replace(r'\s+', ' ', regex=True)
    .str.strip()
    .str.lower()
)
print('Normalized site names:', df['site'].unique())

# Clean up "intervention_group" column
df['intervention_group'] = (
    df['intervention_group'].astype(str)
    .str.replace('_', ' ', regex=False)
    .str.replace(r'\s+', ' ', regex=True)
    .str.strip()
    .str.lower()
)

# Mapping for intervention_group variants
mapping = {
    'contrl': 'control',
    'control': 'control',
    'treatmenta': 'treatment a',
    'treatmen a': 'treatment a',
    'treatment a': 'treatment a',
    'treatment b': 'treatment b',
}
# Apply mapping
df['intervention_group'] = df['intervention_group'].replace(mapping)

print(df['intervention_group'].value_counts())

df.to_csv('output/q6_cleaned_data.csv', index=False)

Normalized site names: ['site b' 'site a' 'site c' 'site d' 'site e']


intervention_group
treatment b    3342
control        3337
treatment a    3321
Name: count, dtype: int64


## Part 4: One-Hot Encoding (5 points)

Create dummy variables for categorical columns:
1. One-hot encode 'intervention_group' using `pd.get_dummies()`
2. One-hot encode 'site'
3. Drop the original categorical columns
4. Show the new shape and column names

In [8]:
# TODO: One-hot encoding
dummies = pd.get_dummies(df, columns=['intervention_group', 'site'])
print(dummies.columns)

new_columns = ['intervention_group_control', 'intervention_group_treatment a',
       'intervention_group_treatment b', 'site_site a', 'site_site b',
       'site_site c', 'site_site d', 'site_site e']
print(dummies[new_columns].describe())


Index(['patient_id', 'age', 'sex', 'bmi', 'enrollment_date', 'systolic_bp',
       'diastolic_bp', 'cholesterol_total', 'cholesterol_hdl',
       'cholesterol_ldl', 'glucose_fasting', 'follow_up_months',
       'adverse_events', 'outcome_cvd', 'adherence_pct', 'dropout',
       'cholesterol_ratio', 'bp_category', 'age_group', 'bmi_category',
       'intervention_group_control', 'intervention_group_treatment a',
       'intervention_group_treatment b', 'site_site a', 'site_site b',
       'site_site c', 'site_site d', 'site_site e'],
      dtype='object')


       intervention_group_control intervention_group_treatment a  \
count                       10000                          10000   
unique                          2                              2   
top                         False                          False   
freq                         6663                           6679   

       intervention_group_treatment b site_site a site_site b site_site c  \
count                           10000       10000       10000       10000   
unique                              2           2           2           2   
top                             False       False       False       False   
freq                             6658        7044        7547        7927   

       site_site d site_site e  
count        10000       10000  
unique           2           2  
top          False       False  
freq          8499        8983  


## Part 5: Save Transformed Data

Save the fully transformed dataset to `output/q6_transformed_data.csv`

In [9]:
# TODO: Save transformed data
dummies.to_csv('output/q6_transformed_data.csv', index=False)
